# Large-Scale Stochastic Variational GP Regression (CUDA)

## Overview

In this notebook, we'll give an overview of how to use SVGP stochastic variational regression ((https://arxiv.org/pdf/1411.2005.pdf)) to rapidly train using minibatches on the `3droad` UCI dataset with hundreds of thousands of training examples. 

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `song` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a **~136 MB** file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('3droad.mat'):
    print('Downloading \'3droad\' UCI dataset...')
    urllib.request.urlretrieve('https://www.dropbox.com/s/f6ow1i59oqx05pl/3droad.mat?dl=1', '3droad.mat')
    
data = torch.Tensor(loadmat('3droad.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Creating a DataLoader

The next step is to create a torch `DataLoader` that will handle getting us random minibatches of data. This involves using the standard `TensorDataset` and `DataLoader` modules provided by PyTorch.

In this notebook we'll be using a fairly large batch size of 1024 just to make optimization run faster, but you could of course change this as you so choose.

In [5]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

## Defining the SVGP Model

We now define the GP regression module that, intuitvely, will act as the final "layer" of our neural network. In this case, because we are doing variational inference and *not* exact inference, we will be using an `AbstractVariationalGP`. In this example, because we will be learning the inducing point locations, we'll be using a base `VariationalStrategy` with `learn_inducing_locations=True`.

Because the feature extractor we defined above extracts two features, we'll need to define our grid bounds over two dimensions.

In [6]:
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import WhitenedVariationalStrategy

class GPModel(AbstractVariationalGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = WhitenedVariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

inducing_points = train_x[:500, :]
model = GPModel(inducing_points=inducing_points).cuda()
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()

## Training the Model

The cell below trains the model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalMarginalLogLikelihood` or ELBO), call backwards, and do a step of optimization.

In [7]:
import time

model.train()
likelihood.train()

# We'll do 6 epochs of training in this tutorial
num_epochs = 4

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 5], gamma=0.1)

# Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0), combine_terms=False)

# We use more CG iterations here because the preconditioner introduced in the NeurIPS paper seems to be less
# effective for VI.
with gpytorch.settings.max_cg_iterations(45):
    for i in range(num_epochs):
        scheduler.step()
        # Within each iteration, we will go over each minibatch of data
        for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):
            start_time = time.time()
            optimizer.zero_grad()
            output = model(x_batch)
            # with combine_terms=False, we get the terms of the ELBO separated so we can print them individually if we'd like.
            # loss = -mll(output, y_batch) would also work.
            log_lik, kl_div, log_prior = mll(output, y_batch)
            loss = -(log_lik - kl_div + log_prior)
            print('Epoch %d [%d/%d] - Loss: %.3f [%.3f, %.3f, %.3f]' % (i + 1, minibatch_i, len(train_loader), loss.item(), log_lik.item(), kl_div.item(), log_prior.item()))

            loss.backward()
            optimizer.step()

Epoch 1 [0/340] - Loss: 284.605 [-284.605, -0.000, 0.000]
Epoch 1 [1/340] - Loss: 407.885 [-407.884, 0.001, 0.000]
Epoch 1 [2/340] - Loss: 266.743 [-266.743, 0.000, 0.000]
Epoch 1 [3/340] - Loss: 314.733 [-314.733, 0.000, 0.000]
Epoch 1 [4/340] - Loss: 354.599 [-354.598, 0.001, 0.000]
Epoch 1 [5/340] - Loss: 313.988 [-313.988, 0.001, 0.000]
Epoch 1 [6/340] - Loss: 269.362 [-269.362, 0.000, 0.000]
Epoch 1 [7/340] - Loss: 251.600 [-251.600, -0.000, 0.000]
Epoch 1 [8/340] - Loss: 278.105 [-278.105, 0.000, 0.000]
Epoch 1 [9/340] - Loss: 254.841 [-254.841, 0.000, 0.000]
Epoch 1 [10/340] - Loss: 288.696 [-288.696, 0.000, 0.000]
Epoch 1 [11/340] - Loss: 236.707 [-236.707, 0.000, 0.000]
Epoch 1 [12/340] - Loss: 230.295 [-230.295, 0.000, 0.000]
Epoch 1 [13/340] - Loss: 212.733 [-212.733, 0.000, 0.000]
Epoch 1 [14/340] - Loss: 238.513 [-238.513, 0.000, 0.000]
Epoch 1 [15/340] - Loss: 241.217 [-241.217, 0.000, 0.000]
Epoch 1 [16/340] - Loss: 214.024 [-214.024, 0.000, 0.000]
Epoch 1 [17/340] - Los

Epoch 1 [144/340] - Loss: 101.966 [-101.948, 0.018, 0.000]
Epoch 1 [145/340] - Loss: 100.822 [-100.804, 0.018, 0.000]
Epoch 1 [146/340] - Loss: 113.399 [-113.381, 0.018, 0.000]
Epoch 1 [147/340] - Loss: 98.197 [-98.178, 0.019, 0.000]
Epoch 1 [148/340] - Loss: 95.927 [-95.908, 0.019, 0.000]
Epoch 1 [149/340] - Loss: 103.746 [-103.726, 0.020, 0.000]
Epoch 1 [150/340] - Loss: 100.094 [-100.074, 0.020, 0.000]
Epoch 1 [151/340] - Loss: 99.724 [-99.703, 0.021, 0.000]
Epoch 1 [152/340] - Loss: 105.135 [-105.114, 0.021, 0.000]
Epoch 1 [153/340] - Loss: 105.083 [-105.062, 0.022, 0.000]
Epoch 1 [154/340] - Loss: 99.960 [-99.938, 0.022, 0.000]
Epoch 1 [155/340] - Loss: 97.062 [-97.040, 0.022, 0.000]
Epoch 1 [156/340] - Loss: 102.908 [-102.885, 0.023, 0.000]
Epoch 1 [157/340] - Loss: 101.583 [-101.560, 0.023, 0.000]
Epoch 1 [158/340] - Loss: 88.308 [-88.285, 0.024, 0.000]
Epoch 1 [159/340] - Loss: 94.407 [-94.383, 0.024, 0.000]
Epoch 1 [160/340] - Loss: 94.089 [-94.065, 0.024, 0.000]
Epoch 1 [161/

Epoch 1 [289/340] - Loss: 71.807 [-71.706, 0.102, 0.000]
Epoch 1 [290/340] - Loss: 67.643 [-67.565, 0.078, 0.000]
Epoch 1 [291/340] - Loss: 72.273 [-72.210, 0.062, 0.000]
Epoch 1 [292/340] - Loss: 60.988 [-60.935, 0.054, 0.000]
Epoch 1 [293/340] - Loss: 67.555 [-67.505, 0.050, 0.000]
Epoch 1 [294/340] - Loss: 69.092 [-69.041, 0.051, 0.000]
Epoch 1 [295/340] - Loss: 63.161 [-63.107, 0.054, 0.000]
Epoch 1 [296/340] - Loss: 64.705 [-64.643, 0.062, 0.000]
Epoch 1 [297/340] - Loss: 64.937 [-64.869, 0.068, 0.000]
Epoch 1 [298/340] - Loss: 65.062 [-64.986, 0.076, 0.000]
Epoch 1 [299/340] - Loss: 60.561 [-60.472, 0.089, 0.000]
Epoch 1 [300/340] - Loss: 56.765 [-56.669, 0.096, 0.000]
Epoch 1 [301/340] - Loss: 64.428 [-64.329, 0.098, 0.000]
Epoch 1 [302/340] - Loss: 62.534 [-62.425, 0.109, 0.000]
Epoch 1 [303/340] - Loss: 68.426 [-68.322, 0.105, 0.000]
Epoch 1 [304/340] - Loss: 60.936 [-60.843, 0.093, 0.000]
Epoch 1 [305/340] - Loss: 57.165 [-57.073, 0.092, 0.000]
Epoch 1 [306/340] - Loss: 57.99

Epoch 2 [95/340] - Loss: 50.599 [-50.499, 0.100, 0.000]
Epoch 2 [96/340] - Loss: 61.430 [-61.327, 0.102, 0.000]
Epoch 2 [97/340] - Loss: 52.454 [-52.350, 0.104, 0.000]
Epoch 2 [98/340] - Loss: 57.813 [-57.703, 0.110, 0.000]
Epoch 2 [99/340] - Loss: 54.270 [-54.157, 0.113, 0.000]
Epoch 2 [100/340] - Loss: 47.857 [-47.741, 0.116, 0.000]
Epoch 2 [101/340] - Loss: 51.680 [-51.559, 0.121, 0.000]
Epoch 2 [102/340] - Loss: 49.377 [-49.252, 0.125, 0.000]
Epoch 2 [103/340] - Loss: 49.895 [-49.764, 0.131, 0.000]
Epoch 2 [104/340] - Loss: 48.714 [-48.577, 0.137, 0.000]
Epoch 2 [105/340] - Loss: 44.537 [-44.395, 0.142, 0.000]
Epoch 2 [106/340] - Loss: 49.458 [-49.316, 0.141, 0.000]
Epoch 2 [107/340] - Loss: 48.410 [-48.270, 0.139, 0.000]
Epoch 2 [108/340] - Loss: 56.088 [-55.952, 0.136, 0.000]
Epoch 2 [109/340] - Loss: 50.461 [-50.328, 0.133, 0.000]
Epoch 2 [110/340] - Loss: 55.284 [-55.155, 0.129, 0.000]
Epoch 2 [111/340] - Loss: 48.117 [-47.995, 0.122, 0.000]
Epoch 2 [112/340] - Loss: 48.796 [-4

Epoch 2 [241/340] - Loss: 48.736 [-48.579, 0.158, 0.000]
Epoch 2 [242/340] - Loss: 57.485 [-57.325, 0.160, 0.000]
Epoch 2 [243/340] - Loss: 43.428 [-43.267, 0.161, 0.000]
Epoch 2 [244/340] - Loss: 51.841 [-51.670, 0.170, 0.000]
Epoch 2 [245/340] - Loss: 43.725 [-43.555, 0.169, 0.000]
Epoch 2 [246/340] - Loss: 41.940 [-41.769, 0.171, 0.000]
Epoch 2 [247/340] - Loss: 46.186 [-46.004, 0.181, 0.000]
Epoch 2 [248/340] - Loss: 54.817 [-54.634, 0.183, 0.000]
Epoch 2 [249/340] - Loss: 46.823 [-46.627, 0.196, 0.000]
Epoch 2 [250/340] - Loss: 51.243 [-51.027, 0.216, 0.000]
Epoch 2 [251/340] - Loss: 68.936 [-68.693, 0.243, 0.000]
Epoch 2 [252/340] - Loss: 50.416 [-50.189, 0.227, 0.000]
Epoch 2 [253/340] - Loss: 53.493 [-53.264, 0.229, 0.000]
Epoch 2 [254/340] - Loss: 48.587 [-48.347, 0.239, 0.000]
Epoch 2 [255/340] - Loss: 53.814 [-53.553, 0.261, 0.000]
Epoch 2 [256/340] - Loss: 58.104 [-57.813, 0.291, 0.000]
Epoch 2 [257/340] - Loss: 52.860 [-52.559, 0.301, 0.000]
Epoch 2 [258/340] - Loss: 67.28

Epoch 3 [46/340] - Loss: 51.017 [-50.779, 0.238, 0.000]
Epoch 3 [47/340] - Loss: 37.612 [-37.402, 0.210, 0.000]
Epoch 3 [48/340] - Loss: 39.810 [-39.617, 0.193, 0.000]
Epoch 3 [49/340] - Loss: 38.108 [-37.911, 0.196, 0.000]
Epoch 3 [50/340] - Loss: 40.810 [-40.621, 0.189, 0.000]
Epoch 3 [51/340] - Loss: 35.829 [-35.639, 0.190, 0.000]
Epoch 3 [52/340] - Loss: 39.424 [-39.230, 0.194, 0.000]
Epoch 3 [53/340] - Loss: 40.097 [-39.902, 0.195, 0.000]
Epoch 3 [54/340] - Loss: 46.032 [-45.834, 0.199, 0.000]
Epoch 3 [55/340] - Loss: 38.829 [-38.641, 0.188, 0.000]
Epoch 3 [56/340] - Loss: 38.512 [-38.325, 0.186, 0.000]
Epoch 3 [57/340] - Loss: 37.688 [-37.516, 0.171, 0.000]
Epoch 3 [58/340] - Loss: 38.388 [-38.220, 0.169, 0.000]
Epoch 3 [59/340] - Loss: 39.455 [-39.297, 0.158, 0.000]
Epoch 3 [60/340] - Loss: 41.008 [-40.850, 0.158, 0.000]
Epoch 3 [61/340] - Loss: 34.305 [-34.146, 0.159, 0.000]
Epoch 3 [62/340] - Loss: 37.840 [-37.674, 0.166, 0.000]
Epoch 3 [63/340] - Loss: 36.268 [-36.096, 0.172,

Epoch 3 [194/340] - Loss: 33.477 [-33.197, 0.280, 0.000]
Epoch 3 [195/340] - Loss: 30.964 [-30.692, 0.271, 0.000]
Epoch 3 [196/340] - Loss: 34.134 [-33.877, 0.257, 0.000]
Epoch 3 [197/340] - Loss: 32.934 [-32.677, 0.257, 0.000]
Epoch 3 [198/340] - Loss: 37.583 [-37.331, 0.251, 0.000]
Epoch 3 [199/340] - Loss: 33.555 [-33.299, 0.256, 0.000]
Epoch 3 [200/340] - Loss: 33.746 [-33.490, 0.256, 0.000]
Epoch 3 [201/340] - Loss: 35.094 [-34.844, 0.250, 0.000]
Epoch 3 [202/340] - Loss: 32.439 [-32.182, 0.256, 0.000]
Epoch 3 [203/340] - Loss: 33.123 [-32.864, 0.259, 0.000]
Epoch 3 [204/340] - Loss: 33.461 [-33.200, 0.260, 0.000]
Epoch 3 [205/340] - Loss: 33.694 [-33.429, 0.264, 0.000]
Epoch 3 [206/340] - Loss: 34.111 [-33.817, 0.294, 0.000]
Epoch 3 [207/340] - Loss: 34.630 [-34.311, 0.319, 0.000]
Epoch 3 [208/340] - Loss: 34.324 [-33.987, 0.337, 0.000]
Epoch 3 [209/340] - Loss: 33.232 [-32.880, 0.352, 0.000]
Epoch 3 [210/340] - Loss: 33.556 [-33.201, 0.355, 0.000]
Epoch 3 [211/340] - Loss: 32.13

Epoch 3 [338/340] - Loss: 28.366 [-27.947, 0.418, 0.000]
Epoch 3 [339/340] - Loss: 26.734 [-26.308, 0.427, 0.000]
Epoch 4 [0/340] - Loss: 29.253 [-28.815, 0.438, 0.000]
Epoch 4 [1/340] - Loss: 26.974 [-26.537, 0.438, 0.000]
Epoch 4 [2/340] - Loss: 25.756 [-25.319, 0.437, 0.000]
Epoch 4 [3/340] - Loss: 28.685 [-28.250, 0.435, 0.000]
Epoch 4 [4/340] - Loss: 26.215 [-25.782, 0.433, 0.000]
Epoch 4 [5/340] - Loss: 27.363 [-26.930, 0.434, 0.000]
Epoch 4 [6/340] - Loss: 28.645 [-28.217, 0.428, 0.000]
Epoch 4 [7/340] - Loss: 29.161 [-28.736, 0.425, 0.000]
Epoch 4 [8/340] - Loss: 27.929 [-27.506, 0.423, 0.000]
Epoch 4 [9/340] - Loss: 26.959 [-26.539, 0.420, 0.000]
Epoch 4 [10/340] - Loss: 25.711 [-25.294, 0.417, 0.000]
Epoch 4 [11/340] - Loss: 25.198 [-24.778, 0.420, 0.000]
Epoch 4 [12/340] - Loss: 28.243 [-27.830, 0.414, 0.000]
Epoch 4 [13/340] - Loss: 27.974 [-27.562, 0.412, 0.000]
Epoch 4 [14/340] - Loss: 27.927 [-27.514, 0.412, 0.000]
Epoch 4 [15/340] - Loss: 27.779 [-27.367, 0.412, 0.000]


Epoch 4 [147/340] - Loss: 26.593 [-26.172, 0.421, 0.000]
Epoch 4 [148/340] - Loss: 27.990 [-27.570, 0.420, 0.000]
Epoch 4 [149/340] - Loss: 27.533 [-27.113, 0.420, 0.000]
Epoch 4 [150/340] - Loss: 25.909 [-25.489, 0.420, 0.000]
Epoch 4 [151/340] - Loss: 26.083 [-25.661, 0.421, 0.000]
Epoch 4 [152/340] - Loss: 26.357 [-25.936, 0.422, 0.000]
Epoch 4 [153/340] - Loss: 27.813 [-27.383, 0.430, 0.000]
Epoch 4 [154/340] - Loss: 25.301 [-24.876, 0.425, 0.000]
Epoch 4 [155/340] - Loss: 24.469 [-24.041, 0.428, 0.000]
Epoch 4 [156/340] - Loss: 26.029 [-25.602, 0.427, 0.000]
Epoch 4 [157/340] - Loss: 28.160 [-27.731, 0.430, 0.000]
Epoch 4 [158/340] - Loss: 29.145 [-28.714, 0.431, 0.000]
Epoch 4 [159/340] - Loss: 25.986 [-25.553, 0.432, 0.000]
Epoch 4 [160/340] - Loss: 26.463 [-26.029, 0.434, 0.000]
Epoch 4 [161/340] - Loss: 27.314 [-26.879, 0.436, 0.000]
Epoch 4 [162/340] - Loss: 22.410 [-21.974, 0.436, 0.000]
Epoch 4 [163/340] - Loss: 23.168 [-22.731, 0.437, 0.000]
Epoch 4 [164/340] - Loss: 27.54

Epoch 4 [291/340] - Loss: 26.725 [-26.281, 0.444, 0.000]
Epoch 4 [292/340] - Loss: 26.811 [-26.369, 0.442, 0.000]
Epoch 4 [293/340] - Loss: 24.623 [-24.181, 0.442, 0.000]
Epoch 4 [294/340] - Loss: 27.364 [-26.918, 0.446, 0.000]
Epoch 4 [295/340] - Loss: 27.307 [-26.864, 0.443, 0.000]
Epoch 4 [296/340] - Loss: 24.420 [-23.978, 0.442, 0.000]
Epoch 4 [297/340] - Loss: 28.476 [-28.033, 0.442, 0.000]
Epoch 4 [298/340] - Loss: 25.343 [-24.900, 0.443, 0.000]
Epoch 4 [299/340] - Loss: 23.952 [-23.508, 0.444, 0.000]
Epoch 4 [300/340] - Loss: 27.814 [-27.368, 0.446, 0.000]
Epoch 4 [301/340] - Loss: 25.337 [-24.892, 0.444, 0.000]
Epoch 4 [302/340] - Loss: 27.782 [-27.336, 0.446, 0.000]
Epoch 4 [303/340] - Loss: 27.219 [-26.775, 0.444, 0.000]
Epoch 4 [304/340] - Loss: 25.723 [-25.280, 0.443, 0.000]
Epoch 4 [305/340] - Loss: 29.251 [-28.809, 0.442, 0.000]
Epoch 4 [306/340] - Loss: 26.144 [-25.702, 0.441, 0.000]
Epoch 4 [307/340] - Loss: 29.418 [-28.976, 0.442, 0.000]
Epoch 4 [308/340] - Loss: 26.50

## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`). Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although this can be done by passing in minibatches of `test_x` rather than the full tensor.

In [11]:
model.eval()
likelihood.eval()
means = torch.tensor([0.])
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = model(x_batch)
        means = torch.cat([means, preds.mean.cpu()])
means = means[1:]

/home/jake.gardner/git/gpytorch/gpytorch/lazy/cached_cg_lazy_tensor.py:161: UserWarning: CachedCGLazyTensor had to run CG on a tensor of size torch.Size([500, 1918]). For best performance, this LazyTensor should pre-register all vectors to run CG against.
  "LazyTensor should pre-register all vectors to run CG against.".format(rhs.shape)


In [13]:
print('Test MAE: {}'.format(torch.mean(torch.abs(means - test_y.cpu()))))

Test MAE: 9.761589050292969
